In [1]:
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime
from pathlib import Path
import os
from os.path import basename
import collections
import tflearn

from collections import deque
from sklearn.model_selection import train_test_split
from tflearn.data_utils import to_categorical
import numpy as np

In [2]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def check_files(folder):
    """
    Given path to folder, returns whether it's empty or not
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    if(len(filenames)==0):
        return False
    return True

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(label2id, label):
    """
    Returns label for a folder
    """
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)
    return X

def get_image_map(folder, label):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    image_map = {}
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        #img_arr = get_img_array(f) / 255.0
        #img_arr = get_img_array(f)
        #img_arr = img_arr.flatten() / 255.0
        img_arr = np.load(f)
        img_arr = img_arr.flatten()
        name = basename(f)
        #print(name)
        num = int(name.split("_")[1].split(".")[0])
        image_map[num] = [img_arr, label]
        #images.append(img_arr)
    #X = np.column_stack(images)
    return image_map

def get_train_data(data_root_path, label_mapping_path):
    """
    Return X and y
    """
    X = []
    y = []
    labels = os.listdir(data_root_path)
    id2label, label2id = get_label_mapping(label_mapping_path)
    print(label2id)
    image_map = {}
    for label in labels:
        train_data_path = data_root_path + label
        if(check_files(train_data_path)) :
            temp_map = get_image_map(train_data_path, label)
            image_map.update(temp_map)
    new_map = collections.OrderedDict(sorted(image_map.items()))
    for k, v in new_map.items():
        X.append(v[0])
        y.append(get_label(label2id, v[1]))
    X = np.array(X)
    return X, np.array(y)

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
# The data folder can have any number of subfolders. 
#The name of the subfolder will be treated as the label for all the images in that subfolder.
#Add all the subfolder names in labels.txt
data_root_path = '/mnt/c/Users/bk262/Desktop/last_layer3/last_layer3/' #will take all the folders in this directory to be used as labels
label_mapping_path = './labels.txt' #labels.txt should NOT be in the data_root_path
X_from_image, y_from_image = get_train_data(data_root_path, label_mapping_path) # this may take a few minutes
print(X_from_image.shape)
#print(X_from_image)
print(y_from_image.shape)
#print(y_from_image)
print('Data loading done')
#print(type(X_from_image))

{'W': 1, 'D': 2, 'A': 0, 'S': 3}
(1714, 128)
(1714,)
Data loading done


In [5]:
def get_data(X_from_image, y_from_image, num_frames, num_classes, input_length):
    X = []
    y = []
    #print(X_from_image.shape)
    #print("hi")
    image_seq = deque()
    #y_list = deque()
    for row in range(0,len(X_from_image)):
        image_seq.append(X_from_image[row])
        #y_list.append(y_from_image[row])
        if len(image_seq) == num_frames:
            X.append(np.array(list(image_seq)))
            y.append(y_from_image[row])
            #y.append(np.array(list(y_list)))
            image_seq.popleft()
            #y_list.popleft()
    
    X = np.array(X)
    y = np.array(y)
    print(X.shape)
    #print(X[0])
    print(y.shape)
    y = to_categorical(y, num_classes)
    return X, y
    

In [6]:
#a = get_data(X_from_image, y_from_image, 4, 4, 128)

In [7]:
def get_rnn(images, input_size, num_classes):
    
    rnn = tflearn.input_data(shape=[None, images, input_size])
    rnn = tflearn.lstm(rnn, 512, dropout=0.8, return_seq=True)
    rnn = tflearn.lstm(rnn, 512)
    rnn = tflearn.fully_connected(rnn, num_classes, activation='softmax')
    rnn = tflearn.regression(rnn, optimizer='adam',
                             loss='categorical_crossentropy', name="output1")
    return rnn

In [8]:
def train(X_train, y_train, seq_length, batch_size):
    
    
    input_length = X_train.shape[2]
    num_classes = len(y_train[0])
    
    rnn = get_rnn(seq_length, input_length, num_classes)

    model = tflearn.DNN(rnn, tensorboard_verbose=0)
    model.fit(X_train, y_train, validation_set=0.1,
              show_metric=True, batch_size=batch_size, snapshot_step=100,
              n_epoch=3)
    #print(model.session)
    #builder = tf.saved_model.builder.SavedModelBuilder('./modelPath1')
    #builder.add_meta_graph_and_variables(model.session, [tf.saved_model.tag_constants.TRAINING],
                                         #signature_def_map=None, assets_collection=None)
    model.save('checkpoints/rnn.tflearn')
    
    #test to predict lable for just one image
    t = []
    t.append(X_train[0])
    print(model.predict_label(t))
    results =  model.predict_label(X_train)
    
    #builder.save()
    return results

In [9]:
if __name__ == '__main__':
    #filename = 'data/cnn-features-frames-2.pkl'
    input_length = 128 #look at the data and figure this out!
    seq_length = 1
    batch_size = 64
    num_classes = 4
    
    X_train, y_train = get_data(X_from_image, y_from_image, seq_length, num_classes, input_length)
    
    results = train(X_train, y_train, seq_length, batch_size)

Training Step: 74  | total loss: 0.32979 | time: 4.478s
| Adam | epoch: 003 | loss: 0.32979 - acc: 0.8649 -- iter: 1536/1542
Training Step: 75  | total loss: 0.33345 | time: 5.663s
| Adam | epoch: 003 | loss: 0.33345 - acc: 0.8626 | val_loss: 0.21259 - val_acc: 0.9535 -- iter: 1542/1542
--
INFO:tensorflow:/root/rnn/checkpoints/rnn.tflearn is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
[[1 0 2 3]]


In [16]:
print(results.shape)
results = np.array(results)
print(len(results[:,0]))
print(len(y_train))

(1714, 4)
1714
1714
